# Étude à l'échelle de la consommation par adresse

Nous tâchons désormais de savoir s'il est possible pour un producteur d'électricité de prévoir la consommation d'électricité *d'une résidence* à partir d'un autre type de données plus local que des données départementales : des données à l'échelle des communes et des adresses. Nous continuons à considérer la météo comme une variable potentiellement pertinente pour comprendre la consommation, et nous cherchons à ajouter à cela la performance énergétique des bâtiments en passant par l'indice de DPE ([Diagnostic de performance énergétique](https://fr.wikipedia.org/wiki/Diagnostic_de_performance_%C3%A9nerg%C3%A9tique)). Ainsi, on cherche à savoir si les producteurs d'électricité, connaissant la notation énergétique d'un bâtiment toutes énergies comprises peuvent en prédire efficacement la consommation électrique de cette adresse au cours de l'année à venir.

Ce notebook s'organise en deux temps. 

- **Premier temps** : Récupération des données et formation d'un data frame regroupant toutes les données intéressantes.   
  Ce premier temps s'organise de la manière suivante :

      1. Récupération des données de consommation électriques individuelles sur le site d'Enedis
      2. Récupération des données météorologiques sur le site d'Infoclimat à l'échelle de communes, at agrégation de ces données pour constituer des moyennes annuelles
      3. Introduction d'une base de données de l'INSEE pour normaliser les noms de commune de la base de données météorologiques
      4. Constitution d'une liste de communes d'étude
      5. Récupération des données DPE
      6. Fusion des différentes données   

    
- **Second temps** : Régression utiles et conclusions (non commencé)

## Introduction - Mise en route

Le plan est le suivant : on cherche à régresser la consommation à l'échelle des adresses en fonction de la météo et de l'indice DPE de l'habitation. Nous cherchons donc une base de données d'adresses avec ces trois caractéristiques. Pour cela, nous allons checher à fusionner trois différentes bases de données :
- Des bases de données de consommations annuelles par adresse sur le site d'[Enedis](https://enedis.opendatasoft.com/explore/dataset/consommation-annuelle-residentielle-par-adresse/information/)
- Des bases de données météorologiques à l'échelle des communes sur le site d'[Infoclimat](https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois)
- Des bases de données de notation DPE par adresse fournie par l'[ADEME (Agence de l'environnement et de la maîtrise de l'énergie)](https://data.ademe.fr/datasets/dpe-france)

Pour permettre la bonne fusion de ces tables, nous importons en plus une base de données de l'INSEE via le module `cartiflette`, pour permettre d'associer à chaque commune son nom officiel et son numéro INSEE de commune.

Les bâtiments sur lesquels portera notre étude seront donc les résidences localisées dans des communes proposant des données météorologiques, et ayant renseignés à la fois leur indice DPE et leur consommation électrique annuelle.

On commence par installer puis importer toutes les librairies et tous les modules qui nous seront utiles.

In [ ]:
!pip install pandas
!pip install geopandas
!pip install geoviews
!pip install lxml
!pip install urllib
!pip install matplotlib
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a
!pip install mapclassify
!pip install folium

In [3]:
!pip install -q lxml

In [4]:
from declarations import *

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 7.17MiB [00:00, 68.9MiB/s]
ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


In [31]:
import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm
import geopandas as gpd

from urllib import request

## 1. Construction d'un tableau de données utile à notre étude 

### 1.1. Récupération des données de consommation électrique individuelle

Pour une étude sur la consommation électrique des bâtiments résidentiels, il paraît raisonnable de commencer par importer des donnés de consommation électrique. À cette fin, on utilise la fonction get_data_consumption, codée dans la page declarations.py , pour récupérer les données de consommation par adresse d'Enedis à partir d'URL, et ce pour quatre années différentes.

In [6]:
import declarations as dec

df_cons_2018=dec.get_data_consumption(dec.consumption_data_url_2018, "2018")
df_cons_2019=dec.get_data_consumption(dec.consumption_data_url_2019, "2019")
df_cons_2020=dec.get_data_consumption(dec.consumption_data_url_2020, "2020")
df_cons_2021=dec.get_data_consumption(dec.consumption_data_url_2021, "2021")

Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.
Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


/home/onyxia/work/Projet_Python/declarations.py:319: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


/home/onyxia/work/Projet_Python/declarations.py:319: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


On affiche ensuite l'une des tables pour montrer ce à quoi ressemblent nos données. On prend l'exemple de l'année 2021.

In [7]:
df_cons_2021.head(4)

,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code INSEE de la commune,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses,Code EPCI,Code Département,Code Région
0,2021,800010101,Centre Ville Émonville,43.0,NaN,RUE,BOUCHER DE PERTHES,80001,Abbeville,RESIDENTIEL,11,59.439,5.404,3.546,43 RUE BOUCHER DE PERTHES,4,200070993.0,80.0,32.0
1,2021,800010502,Delique-Saint-Gilles,2.0,NaN,RUE,D ARTOIS,80001,Abbeville,RESIDENTIEL,31,115.704,3.732,3.546,2 RUE D ARTOIS,9,200070993.0,80.0,32.0
2,2021,800010401,Faubourg de la Bouvaque,30.0,NaN,RUE,DE LA CITE LEDAY,80001,Abbeville,RESIDENTIEL,12,36.454,3.038,3.546,30 RUE DE LA CITE LEDAY,17,200070993.0,80.0,32.0
3,2021,800010502,Delique-Saint-Gilles,8.0,NaN,RUE,DE NORMANDIE,80001,Abbeville,RESIDENTIEL,10,25.769,2.577,3.546,8 RUE DE NORMANDIE,23,200070993.0,80.0,32.0


### 1.2. Scrapping des données météo communales

Une fois les données de consommation électrique récupérées, nous souhaitons nous concentrer sur des données météorologiques à l'échelle la plus locale qu'il nous est possible d'obtenir. Le site Info Climat propose justement des données météorologiques relevées dans plusieurs stations françaises, à une fréquence quotidienne. Par simplicté, nous supposerons que les rélevés météorologiques d'une station portent sur toute la commune dans laquelle est placée la station. 

Les données que nous récupérons sont des moyennes mensuelles des relevés quotidiens. Cependant, nous n'avons accès qu'à des données de consommations individuelles *annuelles*. Pour donner du sens à ces données météorologiques dans notre étude, nous effectuons donc des moyennes annuelles à partir de ces données mensuelles. 

En régressant la consommation par des données météo d'un pas de temps aussi large, on se demande si le "contexte" climatique d'une région ou d'une époque influe sur la consommation d'électricité des ménages. Ce lien pourrait avoir du sens car intuitivement un climat plus froid demande plus de chauffage, et un environnement plus sombre demande plus d'écairage. 

Pour ordonner les différentes fonctions qui se rapportent à l'extraction de données météorologiques à l'échelle des communes, on se propose de construire une classe regroupant les différentes fonctions qui nous intéressent.

In [8]:
import declarations
#Importation de la class Meteo
lameteo=declarations.Meteo()
#On récupère les données
lameteo.scrap()
#On exporte au format DataFrame
lameteo.export()

**Remarque** : On présente ici une rapide parenthèse pour montrer à quel point il est important d'éviter les répétitions de villes et de s'assurer que la station renseigne bien les données météo pour tous les mois de l'année. En effet, dans la cellule ci-dessous, on voit que lorsque qu'on demande le nombre de villes (donc le nombre de stations) avec des informations pour chaque mois, et bien il varie. C'est le cas pour 2019, mais aussi pour toutes les autres années d'études.

In [9]:
[len(lameteo.df_all[2019][i]['ville']) for i in range(1,13)] #Exemple de l'année 2019

[888, 884, 959, 961, 969, 969, 970, 973, 978, 981, 988, 991]

In [10]:
[len(lameteo.df_all[2020][i]['ville']) for i in range(1,13)] #Exemple de l'année 2020

[980, 1432, 1329, 1292, 1320, 1327, 1323, 1329, 1341, 1338, 1346, 1359]

Maintenant que la classe est créée, nous pouvons exporter les données pour voir ce à quoi elles ressemblent. Comme pour les données de consommation, on prend l'exemple de l'année 2021. 

In [11]:
meteo_2021=lameteo.tableau_annuel(2021)
meteo_2021

,Villes,Moyenne des températures minimales extrêmes (\mois),Moyenne des moyennes des températures minimales (\mois),Moyennes des température moyennes (\mois),Moyenne des moyennes des températures maximales (\mois),Moyenne des températures maximales extrêmes (\mois),Moyenne des cumuls de précipitation (\mois),Moyenne des heures d'ensoleillement (\mois),Moyenne des rafales maximales (\mois)
0,Abbeville (80),23.6,88.7,133.5,177.5,264.8,17.4,Rien,Rien
1,Abbeville - centre (80),14.9,87.6,136.1,184.5,270.0,695.8,Rien,460.2
2,ACHERES (78),-4.3,79.3,140.0,200.3,296.2,696.0,Rien,Rien
3,ADAST (65),14.7,82.8,151.4,219.3,325.0,841.5,Rien,Rien
4,Agde (34),79.6,143.3,192.1,240.8,314.3,325.2,Rien,633.9
...,...,...,...,...,...,...,...,...,...
1286,[MAE] Lycée Emile Zola - AIX EN PROVENCE (13),29.9,70.6,131.8,192.9,321.6,576.4,1850.5,477.9
1287,[MAE] Lycée Marguerite Filhol - FUMEL (47),27.0,102.3,160.8,219.4,311.5,891.2,1766.6,576.0
1288,[MAE] Lycée Parc de Vilgénis - MASSY (91),22.9,93.5,147.3,202.1,305.5,Rien,1016.1,506.8
1289,[MAE] Lycée Saint Exupéry - LA ROCHELLE (17),55.2,Rien,Rien,Rien,163.2,447.2,966.9,492.3


### 1.3. Utilisation d'une base intermédiaire pour normaliser les noms de communes

Dans le tableau des données météo précédent, on remarque que certains noms de stations correspondent bien à des noms de ville, mais ça n'est pas le cas de tous. Pour simplifier le croisement avec les autres tableaux de données, nous allons donc importer un tableau de l'INSEE, permettant d'associer quand c'est possible le nom d'une station avec le code INSEE de la commune ainsi que ses données géographiques.

In [12]:
communes = s3.download_vectorfile_url_all(
    values = "metropole",
    crs = 4326,
    borders = "COMMUNE",
    vectorfile_format="topojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

communes["area"] = dep.to_crs(2154).area

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 70.4MiB [00:01, 57.2MiB/s]


In [13]:
communes.head(4)

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,territoire,geometry,area
0,COMMUNE_0000000009754033,NaN,Connangles,CONNANGLES,43076,Commune simple,137,11,1,43,84,200073419,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((3.61421 45.27206, 3.61377 45.27255, ...",5.774291e+09
1,COMMUNE_0000000009760784,NaN,Vidouze,VIDOUZE,65462,Commune simple,243,13,3,65,76,200072106,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-0.02485 43.43002, -0.02500 43.43012...",7.418552e+09
2,COMMUNE_0000000009742077,NaN,Fouesnant,FOUESNANT,29058,Commune simple,9864,11,4,29,53,242900660,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.97912 47.70392, -3.97897 47...",4.033539e+09
3,COMMUNE_0000000009735245,NaN,Plougrescant,PLOUGRESCANT,22218,Commune simple,1166,27,3,22,53,200065928,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.20273 48.84902, -3.20259 48...",4.720673e+09


On donne la liste des colonnes ainsi que le nombres d'éléments distincts que chacune contient pour se faire une idée plus précise de la base.

In [14]:
communes.nunique()

id            34826
ID                0
NOM           32596
NOM_M         32537
INSEE_COM     34826
STATUT            5
POPULATION     5786
INSEE_CAN        56
INSEE_ARR         9
INSEE_DEP        96
INSEE_REG        13
SIREN_EPCI     1244
source            1
territoire        1
geometry      34826
area             96
dtype: int64

### 1.4. Création d'une liste de villes d'étude

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

On voit que les noms de stations ne sont pas très propres : ils incluent les numéros de département, voire incluent des quartiers ou regroupent plusieurs villes. On cherche donc à nettoyer ces noms, pour qu'ils puissent être comparés aux noms de communes donnés par l'API utile pour les DPE. 

On ne touche pour cela qu'à la liste "Villes" du data frame. Nous remplacerons ensuite la base initiale par la base nettoyée, et supprimons les lignes qui ne nous intéressent pas dans le data frame après nettoyage. 

L'API des DPE qui nous intéresse n'identifie une commune que par son code INSEE. Il faut donc que, dans la base de données de chaque station, je réussisse à fusionner la base météo avec une base de données qui comprend les codes communes INSEE à partir de la commune de la station.

Par simplicité, nous ne garderons que les stations dont le nom est un nom de commune officiel ; nous ne chercherons pas pour chaque station la commune à laquelle elle se rapporte.

In [16]:
#On ne garde que les stations dans les villes et pas celles de stations de "Météo à l'école" (MAE)
def sans_mae(year):
    '''Cette fonction supprime les listes du data frame correspondant aux stations MAE
    Entrée : year (type : int, année voulue) 
    Sortie : L[~L['Villes'].str[0].isin(['['])] (type : DataFrame)'''
    L=lameteo.tableau_annuel(year)
    return L[~L['Villes'].str[0].isin(['['])]

Restons cohérents sur nos illustrations et appliquons cette fonction à l'année 2021.

In [17]:
meteo_bis_2021=sans_mae(2021)
meteo_bis_2021

,Villes,Moyenne des températures minimales extrêmes (\mois),Moyenne des moyennes des températures minimales (\mois),Moyennes des température moyennes (\mois),Moyenne des moyennes des températures maximales (\mois),Moyenne des températures maximales extrêmes (\mois),Moyenne des cumuls de précipitation (\mois),Moyenne des heures d'ensoleillement (\mois),Moyenne des rafales maximales (\mois)
0,Abbeville (80),23.6,88.7,133.5,177.5,264.8,17.4,Rien,Rien
1,Abbeville - centre (80),14.9,87.6,136.1,184.5,270.0,695.8,Rien,460.2
2,ACHERES (78),-4.3,79.3,140.0,200.3,296.2,696.0,Rien,Rien
3,ADAST (65),14.7,82.8,151.4,219.3,325.0,841.5,Rien,Rien
4,Agde (34),79.6,143.3,192.1,240.8,314.3,325.2,Rien,633.9
...,...,...,...,...,...,...,...,...,...
1255,Xonrupt-Longemer (88),-35.8,42.5,95.5,148.6,255.1,1428.2,Rien,382.8
1256,Xonrupt-Longemer - Le Poli (88),-27.1,47.3,95.5,143.5,249.1,1906.5,1212.1,346.0
1257,Xonrupt-Longemer - Refuge du Sotré (88),-38.2,37.8,68.7,99.2,207.2,Rien,Rien,942.9
1258,Yvoy-le-Marron (41),-3.5,76.9,137.5,197.7,298.0,85.6,Rien,Rien


On voit qu'en plus par ville il peut il y avoir plusieurs stations (correspondant à plusieurs quartiers). On va donc essayer de supprimer les parties qui ne dénotent pas de la ville dans les noms de stations.

In [18]:
def nettoyage(liste):
    '''Cette fonction est censée enlever les numéros de département des villes dans une liste présentée comme dans les données d'Infoclimat
    Elle met aussi tous les noms de stations en majuscules
    Entrée : liste (type : list ; liste de string) 
    Sortie : List (type : list ; liste de listes)'''
    List=[]
    for ville in liste:
        i=0
        mot=''
        while i < len(ville) and ville[i+1] != '(' :
            mot+=ville[i]
            i+=1
        mot=mot.upper()
        List.append(mot)
    
    return List

def nettoyage_df(year):
    '''Cette fonction est censée séparer les numéros de département des villes dans une liste présentée comme la liste liste_communes
    Entrée : year (type : int ; année désirée) 
    Sortie : L (type : DataFrame ; DataFrame avec les noms actualisés de stations)'''
    L=sans_mae(year)
    L['Villes']=pd.Series(nettoyage(L['Villes'].to_list()))
    return L

Comme pour ce qui précède, on illustre le fonctionnement de cette fonction avec les données de l'année 2021.

In [19]:
meteo_net_2021=nettoyage_df(2021)
meteo_net_2021

,Villes,Moyenne des températures minimales extrêmes (\mois),Moyenne des moyennes des températures minimales (\mois),Moyennes des température moyennes (\mois),Moyenne des moyennes des températures maximales (\mois),Moyenne des températures maximales extrêmes (\mois),Moyenne des cumuls de précipitation (\mois),Moyenne des heures d'ensoleillement (\mois),Moyenne des rafales maximales (\mois)
0,ABBEVILLE,23.6,88.7,133.5,177.5,264.8,17.4,Rien,Rien
1,ABBEVILLE - CENTRE,14.9,87.6,136.1,184.5,270.0,695.8,Rien,460.2
2,ACHERES,-4.3,79.3,140.0,200.3,296.2,696.0,Rien,Rien
3,ADAST,14.7,82.8,151.4,219.3,325.0,841.5,Rien,Rien
4,AGDE,79.6,143.3,192.1,240.8,314.3,325.2,Rien,633.9
...,...,...,...,...,...,...,...,...,...
1255,XONRUPT-LONGEMER,-35.8,42.5,95.5,148.6,255.1,1428.2,Rien,382.8
1256,XONRUPT-LONGEMER - LE POLI,-27.1,47.3,95.5,143.5,249.1,1906.5,1212.1,346.0
1257,XONRUPT-LONGEMER - REFUGE DU SOTRÉ,-38.2,37.8,68.7,99.2,207.2,Rien,Rien,942.9
1258,YVOY-LE-MARRON,-3.5,76.9,137.5,197.7,298.0,85.6,Rien,Rien


On cherche à savoir combien de villes on est amenés à garder si l'on ne veut pas trop se casser la tête et ne garder que les noms "jolis" de stations, pour lesquels on peut facilement trouver le nom de la ville d'origine.

In [20]:
def fusion_commune(year):
    '''Cette fonction croise le tableau des noms de commune avec le tableau des données météo pour une année donnée
    Le croisement est fait à partir du nom de la commune ; on exclut donc les stations qui n'ont pas de noms de commune "propres"
    Entrée : year (type : int ; année désirée) 
    Sortie : filtre_conso (type : DataFrame ; DataFrame de croisement des données)'''
    L=nettoyage_df(year)
    filtre=L.merge(communes, left_on='Villes', right_on='NOM_M')[['NOM_M','STATUT','POPULATION','INSEE_COM','geometry']]
    return filtre

À nouveau, nous y joignons une illustration avec l'année 2021.

In [21]:
filtre_2021=fusion_commune(2021)
filtre_2021

,NOM_M,STATUT,POPULATION,INSEE_COM,geometry
0,ABBEVILLE,Sous-préfecture,22980,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ..."
1,ACHERES,Commune simple,21529,78005,"POLYGON ((2.04953 48.95702, 2.04938 48.95770, ..."
2,ACHERES,Commune simple,380,18001,"POLYGON ((2.44015 47.27926, 2.44015 47.27965, ..."
3,ADAST,Commune simple,297,65001,"POLYGON ((-0.06867 42.97044, -0.07087 42.97014..."
4,AGDE,Commune simple,29600,34003,"MULTIPOLYGON (((3.50101 43.26349, 3.50130 43.2..."
...,...,...,...,...,...
751,WINTERSBOURG,Commune simple,269,57747,"POLYGON ((7.19741 48.77898, 7.19668 48.77917, ..."
752,XERTIGNY,Commune simple,2586,88530,"POLYGON ((6.45837 48.04351, 6.45807 48.04126, ..."
753,XONRUPT-LONGEMER,Commune simple,1515,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ..."
754,YVOY-LE-MARRON,Commune simple,760,41297,"POLYGON ((1.84782 47.58568, 1.84724 47.58529, ..."


Après ce filtre, il nous reste donc par exemple 756 communes d'études pour l'année 2021, ce qui reste satisfaisant : on est rassuré que notre tri très simple n'ait pas enlevé trop de villes d'étude.

Maintenant, nous fusionnons cette base triée de données météo avec la base des données de consommation.

In [22]:
import declarations as dec

def croisement_meteo_conso(year):
    '''Cette fonction croise le tableau des données météo avec le tableau des consommations par adresse pour une année donnée
    Le croisement est fait à partir du nom de la commune
    Attention, l'année renseignée doit nécessairement faire partie des années pour lesquelles on a enregistré des données de consommation 
    dans le fichier declarations, c'est-à-dire que "year" doit valoir 2018, 2019, 2020 ou 2021.
    
    Entrée : year (type : int ; année désirée) 
    Sortie : filtre_conso (type : DataFrame ; DataFrame de croisement des données)'''
    fichier=getattr(dec,"consumption_data_url_"+str(year))
    conso =dec.get_data_consumption(fichier, str(year))
    L=conso['Nom de la commune'].to_list()
    conso['Nom de la commune']=pd.Series([x.upper() for x in L])
    filtre=fusion_commune(year)
    filtre_conso=filtre.merge(conso, left_on="NOM_M", right_on="Nom de la commune")[['NOM_M','STATUT','INSEE_COM','geometry','Code IRIS', 'Nom IRIS', 'Nombre de logements', "Consommation annuelle totale de l'adresse (MWh)", "Consommation annuelle moyenne par logement de l'adresse (MWh)", "Consommation annuelle moyenne de la commune (MWh)", "Adresse"]]   
    return filtre_conso

Une nouvelle illustration avec l'année 2021.

In [23]:
ff_2021=croisement_meteo_conso(2021)
ff_2021

,NOM_M,STATUT,INSEE_COM,geometry,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010101,Centre Ville Émonville,11,59.439,5.404,3.546,43 RUE BOUCHER DE PERTHES
1,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,31,115.704,3.732,3.546,2 RUE D ARTOIS
2,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010401,Faubourg de la Bouvaque,12,36.454,3.038,3.546,30 RUE DE LA CITE LEDAY
3,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,10,25.769,2.577,3.546,8 RUE DE NORMANDIE
4,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,10,22.995,2.299,3.546,20 RUE DE NORMANDIE
...,...,...,...,...,...,...,...,...,...,...,...
35160,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),52,110.382,2.123,5.875,224 CHEMIN DE L HERMINE
35161,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),10,20.775,2.078,5.875,478 ROUTE DES PERGIS
35162,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),14,85.812,6.129,5.875,209 ROUTE DES PERGIS
35163,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),12,37.800,3.150,5.875,65 IMPASSE DU GRAND RAIN


In [24]:
len(ff_2021['NOM_M'].unique()) #Nombre de noms de communes dans notre tableau en 2021

249

In [25]:
def liste_codes_commune(year):
    '''
    Cette fonction sert à dresser une liste des codes INSEE des communes présentes dans notre DataFrame final
    
    Entrée : year (type : int ; année désirée) 
    Sortie : L['INSEE_COM'].unique() (type : array ; Liste des codes communes)'''
    L =croisement_meteo_conso(year)
    return L['INSEE_COM'].unique()

In [26]:
liste_communes_2021=liste_codes_commune(2021) #Nombre de communes réellement comptées dans le tableau en 2021
len(liste_communes_2021)

322

Attention ! On remarque ici que notre base de données comporte beaucoup de villes homonymes. Il faudrait donc bien faire attention au code INSEE / au code postal d'une ville pour s'assurer qu'on ne la confond pas avec une autre ville du même nom.

Notre étude portera donc au plus sur des immeubles issus de 249 communes différentes !

On essaiera par la suite de trouver un moyen de relier l'adresse de la base de données des DPE à l'adresse de la base qu'on vient dd définir. Pour cela, on cherchera à normaliser l'adresse de notre base.

In [27]:
def croisement_meteo_conso_adresse(year):
    '''Cette fonction ajoute une colonne avec adresse au format semblable à celui des DPE au DataFrame de la dernière fonction
    Entrée : year (type : int ; année désirée) 
    Sortie : filtre_conso (type : DataFrame ; DataFrame de croisement des données avec une colonne en plus de croisement_meteo_conso(year))'''
    F=croisement_meteo_conso(year)
    F['Adresse normalisée']=F['Adresse']+ ' '+ F['NOM_M']
    return F

Comme ce qui précède, une visualisation avec 2021.

In [28]:
filtre_conso_adresse_2021=croisement_meteo_conso_adresse(2021)
filtre_conso_adresse_2021

,NOM_M,STATUT,INSEE_COM,geometry,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Adresse normalisée
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010101,Centre Ville Émonville,11,59.439,5.404,3.546,43 RUE BOUCHER DE PERTHES,43 RUE BOUCHER DE PERTHES ABBEVILLE
1,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,31,115.704,3.732,3.546,2 RUE D ARTOIS,2 RUE D ARTOIS ABBEVILLE
2,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010401,Faubourg de la Bouvaque,12,36.454,3.038,3.546,30 RUE DE LA CITE LEDAY,30 RUE DE LA CITE LEDAY ABBEVILLE
3,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,10,25.769,2.577,3.546,8 RUE DE NORMANDIE,8 RUE DE NORMANDIE ABBEVILLE
4,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,10,22.995,2.299,3.546,20 RUE DE NORMANDIE,20 RUE DE NORMANDIE ABBEVILLE
...,...,...,...,...,...,...,...,...,...,...,...,...
35160,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),52,110.382,2.123,5.875,224 CHEMIN DE L HERMINE,224 CHEMIN DE L HERMINE XONRUPT-LONGEMER
35161,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),10,20.775,2.078,5.875,478 ROUTE DES PERGIS,478 ROUTE DES PERGIS XONRUPT-LONGEMER
35162,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),14,85.812,6.129,5.875,209 ROUTE DES PERGIS,209 ROUTE DES PERGIS XONRUPT-LONGEMER
35163,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),12,37.800,3.150,5.875,65 IMPASSE DU GRAND RAIN,65 IMPASSE DU GRAND RAIN XONRUPT-LONGEMER


### 1.5. Récupération des données de Diagnostics de Performance Energétique (DPE)

Il est trop coûteux de récupérer l'intégralité des données de DPE, et nous ne disposons de données météo que pour certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de les agréger dans une unique table de données.

On suppose (hypothèse forte !) que le DPE est intemporel, c'est-à-dire qu'il est une caractéristique intrinsèque du bâtiment : on ne s'intéresse pas à l'année d'évaluation du logement.

In [32]:
def get_json_results(data, code_commune):
    '''Cette fonction ajoute à une liste de données DPE au format jason les données correspondant à la ville de code INSEE "code_commune"
        Entrée : 
            data (type : list, liste de données au format jason)
            code_commune (type : int, code INSEE de la commune dont on veut ajouter les données)
        Sortie : 
            updated_data (type : list, liste de données au format jason)
    '''
    updated_data = data.copy()
    api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
    url_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + "&select=" + "%2A&sampling=neighbors"
    req=requests.get(url_api)
    wb=req.json()
    updated_data += wb['results']
    return updated_data

def api_ademe(year):
    '''Cette fonction extrait les données de l'API des DPE pour les bâtiments localisés dans les communes dont les codes INSEE sont présentés 
    dans la liste "codes_insee"
        Entrée : 
            year (type : int, année désirée)
        Sortie : 
            dpe (type : DataFrame, tableau des données extraites)
    '''
    data=[]
    codes_insee= liste_codes_commune(year)
    for ville in codes_insee :
        data=get_json_results(data, ville)
    df = pd.json_normalize(data)
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

On illustre à nouveau avec l'année 2021, en reprenant la liste que nous avons construite dans le paragraphe 1.4.

In [33]:
api=api_ademe(2021)
api.head(4)

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,classe_estimation_ges,version_methode_dpe,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,geometry
0,F,Location,1974,"50.105335,1.832982",50.105335,100.00,54,Maison Individuelle,12 Place Max Lejeune 80100 Abbeville,786615,...,D,V2012,3CL - DPE,35,430.00,2013-05-06,1.832982,None,Odq6IjxU8z7myJ_DsWJSf,POINT (1.83298 50.10533)
1,D,Vente,1948,"50.095824,1.841555",50.095824,90.00,72,Maison Individuelle,20 Rue de la Plume 80100 Abbeville,27716,...,E,V2012,FACTURE - DPE,80,230.43,2013-05-02,1.841555,None,6th3VqWvwTTZ2doeoKEhc,POINT (1.84156 50.09582)
2,D,Location,1982,"50.109299,1.842122",50.109299,42.80,78,Logement,2 Avenue Aristide Briand 80100 Abbeville,221098,...,B,V2012,3CL - DPE,80,223.39,2013-05-03,1.842122,None,FUt8L1k-W2VQNuouuQbAO,POINT (1.84212 50.10930)
3,N,Vente,1974,"50.109172,1.806312",50.109172,182.05,220,Maison Individuelle,246 Chaussée de Rouvroy 80100 Abbeville,680450,...,N,V2012,3CL - DPE,80,0.00,2013-05-23,1.806312,None,36oVtbKpj76C9vKC-DHcX,POINT (1.80631 50.10917)


In [34]:
api["code_insee_commune_actualise"].nunique()

316

### 1.6 Fusion des tableaux

Il ne nous reste enfin plus qu'à fusionner les tableaux pour obtenir dans une même ligne toutes les données qui nous semblent importantes, à savoir les données de consommations, les données DPE et les données météorologiques annuelles. 

Nous pensions dans un premier temps fusionner les deux tableaux à partir de l'adresse postale de chaque résidence : cette adresse est présente dans le DataFrame des DPE, et peut être presque reconduite dans le DataFrame des consommations. Le "presque" vient ici du fait qu'une adresse postale utilise le code *postal*, alors que le tableau des données de consommation ne nous donne que le tableau des codes *INSEE* des communes. Il aurait donc fallu passer par une base de données intermédiaire, téléchargeable depuis le site de [La Poste](https://datanova.laposte.fr/datasets/laposte-hexasmal) (ou accessible par une API), pour convertir les codes INSEE en codes postaux. Cependant, l'API demande une inscription sur le site de La Poste, et l'autre alternative consiste à télécharger le fichier, ce qui pose un souci pour la reproductibilité simple de nos manipulations. 

Nous avons donc finalement pensé à découper les adresses de la base de données des API en "nom de ville" + "nom de rue", et donc juste passer outre le code postal. Nous prenons cependant un risque, car nous avons vu que les villes qui nous intéressent dans la base de 2021 par exemple étaient susceptibles d'avoir des homonymes. Ceci pourrait donc poser un problème si, dans deux villes homonymes, une même adresse était renseignée, mais nous allons supposer que cette situation est improbable et qu'elle serait un grand coup de malchance.


In [35]:
def reformatage_date(year):
    '''
    Cette fonction crée un DataFrame avec une nouvelle colonne, proposant un format avec adresse en majuscule et pas de code postal

    Entrée : 
            year (type : int, année désirée)
        Sortie : 
            df_year (type : DataFrame, tableau des données extraites)
    '''
    df_year=api_ademe(year)
    liste=list(df_year['geo_adresse'])
    nombres=[str(i) for i in range(0,10)]
    U=[]
    for l in liste :
        L=list(l)
        i=0
        u=len(L)
        while i < (len(L)-5) and u==len(L):
            print (i,L)
            if L[i] in nombres and L[i+1] in nombres and L[i+2] in nombres and L[i+3] in nombres and L[i+4] in nombres :
                for j in range(6) :
                    L.pop(i)
            i+=1
        mot=''
        for letter in L :
            mot+=letter
        mot=mot.upper()
        U.append(mot)
    df_year['Belle adresse'] =pd.Series(U)
    return df_year

Pour l'année 2021 : 

In [36]:
nouveau_2021=reformatage_date(2021)
nouveau_2021

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,version_methode_dpe,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,geometry,Belle adresse
0,F,Location,1974,"50.105335,1.832982",50.105335,100.00,54,Maison Individuelle,12 Place Max Lejeune 80100 Abbeville,786615,...,V2012,3CL - DPE,35,430.00,2013-05-06,1.832982,None,Odq6IjxU8z7myJ_DsWJSf,POINT (1.83298 50.10533),12 PLACE MAX LEJEUNE ABBEVILLE
1,D,Vente,1948,"50.095824,1.841555",50.095824,90.00,72,Maison Individuelle,20 Rue de la Plume 80100 Abbeville,27716,...,V2012,FACTURE - DPE,80,230.43,2013-05-02,1.841555,None,6th3VqWvwTTZ2doeoKEhc,POINT (1.84156 50.09582),20 RUE DE LA PLUME ABBEVILLE
2,D,Location,1982,"50.109299,1.842122",50.109299,42.80,78,Logement,2 Avenue Aristide Briand 80100 Abbeville,221098,...,V2012,3CL - DPE,80,223.39,2013-05-03,1.842122,None,FUt8L1k-W2VQNuouuQbAO,POINT (1.84212 50.10930),2 AVENUE ARISTIDE BRIAND ABBEVILLE
3,N,Vente,1974,"50.109172,1.806312",50.109172,182.05,220,Maison Individuelle,246 Chaussée de Rouvroy 80100 Abbeville,680450,...,V2012,3CL - DPE,80,0.00,2013-05-23,1.806312,None,36oVtbKpj76C9vKC-DHcX,POINT (1.80631 50.10917),246 CHAUSSÉE DE ROUVROY ABBEVILLE
4,D,Vente,1974,"50.108204,1.837013",50.108204,146.80,230,Logement,27 Chaussée du Bois 80100 Abbeville,531806,...,V2012,3CL - DPE,80,189.78,2013-05-15,1.837013,None,XqBbYI7CzktOxEVb_Ve_T,POINT (1.83701 50.10820),27 CHAUSSÉE DU BOIS ABBEVILLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3616,F,Vente,1981,"45.70891,4.585596",45.708910,135.00,111488,Maison Individuelle,7 Montée des Bruyères 69510 Yzeron,67993,...,V2012,3CL - DPE,69,344.65,2016-09-12,4.585596,None,hxx7fyEt3W4zewpeyiYyj,POINT (4.58560 45.70891),NaN
3617,B,Neuf,2016,"45.709107,4.586496",45.709107,122.22,118981,Maison Individuelle,Montée des Bruyères 69510 Yzeron,326477,...,V2012,FACTURE - DPE,69,63.52,2016-11-15,4.586496,None,_hwrXzPh80o77KewGC0s6,POINT (4.58650 45.70911),NaN
3619,D,Vente,1948,"45.718229,4.605208",45.718229,115.00,219700,Maison Individuelle,2294 Route de la Brally 69510 Yzeron,148079,...,V2012,FACTURE - DPE,69,210.95,2019-01-10,4.605208,None,0pQl2zo6d3NOP1ZIryNc6,POINT (4.60521 45.71823),NaN
3620,B,Neuf,2019,"45.705667,4.591846",45.705667,133.43,259989,Maison Individuelle,Chemin du Planil 69510 Yzeron,808398,...,V2012,FACTURE - DPE,69,68.45,2019-10-11,4.591846,None,gquBhuTROFmG2jdRDi_II,POINT (4.59185 45.70567),NaN


In [37]:
def fusion_finale(year):
    '''
    Cette fonction crée un DataFrame qui fusionne toutes les données voulues (consommation, température et indice DPE)

    Entrée : 
            year (type : int, année désirée)
        Sortie : 
            df_year (type : DataFrame, tableau des données finales)
    '''
    conso_propre=croisement_meteo_conso_adresse(year)
    dpe_propre=reformatage_date(year)
    D=conso_propre.merge(dpe_propre, left_on='Adresse normalisée', right_on="Belle adresse")
    return D

On conclut, encore et toujours, avec l'année 2021 : 

In [38]:
fusion_finale_2021=fusion_finale(2021)
fusion_finale_2021

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,NOM_M,STATUT,INSEE_COM,geometry_x,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),...,version_methode_dpe,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,geometry_y,Belle adresse
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010401,Faubourg de la Bouvaque,75,429.267,5.724,3.546,...,V2012,3CL - DPE,80,223.39,2013-05-03,1.842122,None,FUt8L1k-W2VQNuouuQbAO,POINT (1.84212 50.10930),2 AVENUE ARISTIDE BRIAND ABBEVILLE
1,AGDE,Commune simple,34003,"MULTIPOLYGON (((3.50101 43.26349, 3.50130 43.2...",340030201,Le Cap d'Agde,144,189.284,1.314,2.688,...,V2012,FACTURE - DPE,34,44.29,2013-06-27,3.449652,None,DUGBp4PZvW2F6VH42WwqO,POINT (3.44965 43.29055),4 AVENUE DES CANTINIERES AGDE
2,AIGUES-MORTES,Commune simple,30003,"POLYGON ((4.24417 43.50143, 4.24402 43.50133, ...",300030102,Périphérie,37,150.830,4.076,5.817,...,V2012,3CL - DPE,30,147.60,2013-06-20,4.180807,None,9I0j7XA2OvcfoBxNTikgU,POINT (4.18081 43.57410),32 PLACE DE VERDUN AIGUES-MORTES
3,AIX-LES-BAINS,Commune simple,73008,"POLYGON ((5.87119 45.70794, 5.88942 45.71350, ...",730080201,Mémard-Corsuet,14,85.233,6.088,3.413,...,V2012,3CL - DPE,73,279.37,2013-06-10,5.913070,None,zlqZm7I3ymdiSiu8TyYCw,POINT (5.91307 45.68550),20 RUE JULES PIN AIX-LES-BAINS
4,AIX-LES-BAINS,Commune simple,73008,"POLYGON ((5.87119 45.70794, 5.88942 45.71350, ...",730080102,Centre Ville-Sud,45,93.455,2.077,3.413,...,V2012,3CL - DPE,73,311.53,2013-05-22,5.907386,None,kIDtA4oN8biJXqvtLtnfK,POINT (5.90739 45.68997),8 AVENUE DE MARLIOZ AIX-LES-BAINS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,VERNEUIL-SUR-SEINE,Commune simple,78642,"POLYGON ((1.95353 48.97009, 1.95309 48.97077, ...",786420101,Les Hauts de Verneuil,20,31.731,1.587,4.895,...,v15c,3CL,17,227.87,2013-07-30,-0.635495,None,YQKsXinic4qIRriLpCMON,POINT (-0.63550 45.74775),3 ALLEE DES CAPUCINES VERNEUIL-SUR-SEINE
172,VERNEUIL-SUR-SEINE,Commune simple,78642,"POLYGON ((1.95353 48.97009, 1.95309 48.97077, ...",786420103,Richelieu,16,28.800,1.800,4.895,...,V2012,3CL - DPE,83,188.60,2013-04-17,5.809236,None,rzAlckUZWhZ7v3rZOL7sM,POINT (5.80924 43.12161),11 RUE JEAN ZAY VERNEUIL-SUR-SEINE
173,VILLEFONTAINE,Commune simple,38553,"POLYGON ((5.12450 45.60784, 5.12450 45.60794, ...",385530104,Les Picotières-Le Village,80,957.464,11.968,6.574,...,V2012,3CL - DPE,21,123.54,2014-08-18,4.221516,None,uwEuQStJbo70OV32ByVVb,POINT (4.22152 47.27517),7 RUE DU SORBIER VILLEFONTAINE
174,VILLEMOISSON-SUR-ORGE,Commune simple,91667,"POLYGON ((2.33812 48.66757, 2.33944 48.66757, ...",916670000,Villemoisson-sur-Orge (commune non irisée),10,122.109,12.211,7.238,...,V2012,3CL - DPE,54,309.27,2013-10-17,6.264390,None,s3tiKfL18njgrQa3cvsI3,POINT (6.26439 48.72070),19 RUE DU REPOS VILLEMOISSON-SUR-ORGE


Dans cette partie, nous avons donc construit l'ensemble de données utile pour une régression des données de consommations électriques annuelles individuelles des résidences en fonction de l'indice DPE de la résidence, et des données météo annuelles de la commune de la résidence.

## 2. Régression

Nous n'avons malheuresement pas eu le temps d'arriver jusqu'à cette partie. À partir de la base de données concentrant toutes les données que nous avons construite dans la partie 1, nous aurions souhaité réaliser des régressions comme dans l'étude à l'échelle départementale pour conclure ou non sur la pertinence 